In [1]:
from tqdm import tqdm
from sklearn.decomposition import NMF
import pandas as pd
from scipy import sparse
import numpy as np

# read data
movies = pd.read_csv("data/ml-20m/movies.csv")
ratings = pd.read_csv("data/ml-20m/ratings.csv")

# join
ratings_joined = pd.merge(ratings, movies)

# ratingsをsparse matrixに変換して横持ちにする
action_adventure_ratings = ratings_joined.query("genres.str.contains('Action') or genres.str.contains('Adventure')", 
                                                engine='python').reset_index(drop=True)
# indexing ids
# userid
userid_unique = pd.Series(action_adventure_ratings["userId"].unique())
index_userid_dict = userid_unique.to_dict()
# inverse
userid_index_dict = dict(map(reversed, index_userid_dict.items()))

# itemid
itemid_unique = pd.Series(action_adventure_ratings["movieId"].unique())
index_itemid_dict = itemid_unique.to_dict()
# inverse
itemid_index_dict = dict(map(reversed, index_itemid_dict.items()))

action_adventure_ratings["user_id"] = action_adventure_ratings["userId"].map(userid_index_dict)
action_adventure_ratings["item_id"] = action_adventure_ratings["movieId"].map(itemid_index_dict)

# reindexしたidを使って、アイテムとジャンルの対応が取れるdictを作る
itemid_genres_dict = action_adventure_ratings[['item_id', 'genres']].set_index('item_id')['genres'].to_dict()

In [2]:
import cloudpickle
X_train = cloudpickle.load(open("output/ML-20M-X_train.pkl","rb"))
X_test = cloudpickle.load(open("output/ML-20M-X_test.pkl","rb"))

In [3]:
# aggregateのtrainをactionとadventureに分離する
# actionの列
action_columns = [v for v in range(X_train.shape[1]) if 'Action' in itemid_genres_dict[v]]
# adventureの列
adventure_columns = [v for v in range(X_train.shape[1]) if 'Adventure' in itemid_genres_dict[v]]

# 選んだカラムに応じてとってくる
action_train = X_train[:, action_columns]
adventure_train = X_train[:, adventure_columns]

# adventureのみ、アイテムidのconcatとの対応関係が必要なので辞書として持っておく
adventure_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'Adventure' in itemid_genres_dict[v]:
        adventure_concat_itemid_dict[v] = count
        count += 1

In [4]:
# アイテムidのconcatとの対応関係が必要なので辞書として持っておく
action_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'Action' in itemid_genres_dict[v]:
        action_concat_itemid_dict[v] = count
        count += 1
# inverse
inverse_action_concat_itemid_dict = dict(map(reversed, action_concat_itemid_dict.items()))

adventure_concat_itemid_dict = {}
count = 0
for v in range(X_train.shape[1]):
    if 'Adventure' in itemid_genres_dict[v]:
        adventure_concat_itemid_dict[v] = count
        count += 1
# inverse
inverse_adventure_concat_itemid_dict = dict(map(reversed, adventure_concat_itemid_dict.items()))

In [5]:
# それぞれにアクションしていないユーザを削る
# 全ユーザと、削ったあとでの対応関係を辞書として持っておく
action_train_selected = action_train[action_train.getnnz(1)>0]
adventure_train_selected = adventure_train[adventure_train.getnnz(1)>0]

action_train_action_users = {}
action_users = action_train.getnnz(1)>0
count = 0
for i in range(action_train.shape[0]):
    if action_users[i]:
        action_train_action_users[i] = count
        count += 1

# inverse
inverse_action_train_action_users = dict(map(reversed, action_train_action_users.items()))

adventure_train_action_users = {}
adventure_users = adventure_train.getnnz(1)>0
count = 0
for i in range(adventure_train.shape[0]):
    if adventure_users[i]:
        adventure_train_action_users[i] = count
        count += 1

# inverse
inverse_adventure_train_action_users = dict(map(reversed, adventure_train_action_users.items()))

In [6]:
# それぞれでALSする
# 今回は mediateでやったときのものを使う
action_ALS = cloudpickle.load(open('output/ML-20M-action_ALS.pkl', 'rb'))
adventure_ALS = cloudpickle.load(open("output/ML-20M-adventure_ALS.pkl","rb"))

action_ALS_user_vectors = action_ALS.user_factors
adventure_ALS_user_vectors = adventure_ALS.user_factors

In [7]:
# actionとadventureでoverlapしているユーザで、ベクトルの対応表を作る
overlap_action_user_vectors = []
overlap_adventure_user_vectors = []
count = 0
for u in tqdm(range(X_train.shape[0])):
    if u in action_train_action_users and u in adventure_train_action_users:
        overlap_action_user_vectors.append(action_ALS_user_vectors[action_train_action_users[u]].tolist())
        overlap_adventure_user_vectors.append(adventure_ALS_user_vectors[adventure_train_action_users[u]].tolist())

100%|██████████| 138389/138389 [00:02<00:00, 53248.27it/s]


In [8]:
# CCAの学習をする
from sklearn.cross_decomposition import CCA

np.random.seed(42)
cca = CCA(n_components=100)
cca.fit(overlap_action_user_vectors, overlap_adventure_user_vectors)

/home/fufufukakaka/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:77: UserWarning: Maximum number of iterations reached
  warnings.warn('Maximum number of iterations reached')


CCA(copy=True, max_iter=500, n_components=100, scale=True, tol=1e-06)

In [9]:
# 評価対象のユーザ
test_adventure_pos_items_dict = {}
for i in tqdm(range(X_test.shape[0])):
    # trainでadventureにアクションしていないユーザに
    rated_items = X_train[i, :].indices
    if len([v for v in rated_items if 'Adventure' in itemid_genres_dict[v]]) == 0:
        # X_testの中でstoreしているアイテムが0以上のユーザに
        if X_test[i, :].nnz > 0:
            test_items = []
            selected_user_ratings = X_test[i, :]
            value_indices = selected_user_ratings.indices
            sorted_indices = np.argsort(-X_test[i, :].toarray())[0]
            # valueがあるアイテムのジャンルがadventureの場合に
            for v in sorted_indices[:len(value_indices)]:
                if 'Adventure' in itemid_genres_dict[v]:
                    test_items.append(v)
            if len(test_items) > 0:
                test_adventure_pos_items_dict[i] = test_items

100%|██████████| 138389/138389 [00:21<00:00, 6419.33it/s]


In [10]:
# adventureのitemのベクトル
adventure_item_vectors = adventure_ALS.item_factors

In [11]:
from lib.recommend_util import ndcg
ndcgs = {
    'ndcg5':  [],
    'ndcg10':  [],
    'ndcg20':  [],
    'ndcg50':  [],
    'ndcg100':  []
}
count = 0
for userid, pos_items in tqdm(test_adventure_pos_items_dict.items()):
   # pos_itemsをadventure_matrixの次元に変換する
    pos_items = np.array([adventure_concat_itemid_dict[v] for v in pos_items])
    # useridに対応するユーザベクトル(action)を得る
    try:
        action_userid = action_train_action_users[userid]
        action_user_vector = action_ALS_user_vectors[action_userid, :]
        # CCAを使ってadventureの次元に変換する
        adventure_user_vector_action_CCA = action_user_vector @ cca.x_weights_ @ cca.y_weights_.T
        # adventureのitemのベクトルと掛け合わせる
        adv_predict = np.dot(adventure_user_vector_action_CCA, adventure_item_vectors.T)
        # sum_ratingsをargsort
        sorted_indices = np.array([v for v in np.argsort(-adv_predict)])
        ndcgs['ndcg5'].append(ndcg(sorted_indices[:5], pos_items))
        ndcgs['ndcg10'].append(ndcg(sorted_indices[:10], pos_items))
        ndcgs['ndcg20'].append(ndcg(sorted_indices[:20], pos_items))
        ndcgs['ndcg50'].append(ndcg(sorted_indices[:50], pos_items))
        ndcgs['ndcg100'].append(ndcg(sorted_indices[:100], pos_items))
    except:
        count += 1
        # 推薦できないユーザの場合は無条件で0を入れる
        ndcgs['ndcg5'].append(0)
        ndcgs['ndcg10'].append(0)
        ndcgs['ndcg20'].append(0)
        ndcgs['ndcg50'].append(0)
        ndcgs['ndcg100'].append(0)

100%|██████████| 18523/18523 [00:50<00:00, 364.51it/s]


In [13]:
print("ndcg@5: {}".format(np.mean(ndcgs['ndcg5'])))
print("ndcg@10: {}".format(np.mean(ndcgs['ndcg10'])))
print("ndcg@20: {}".format(np.mean(ndcgs['ndcg20'])))
print("ndcg@50: {}".format(np.mean(ndcgs['ndcg50'])))
print("ndcg@100: {}".format(np.mean(ndcgs['ndcg100'])))

ndcg@5: 0.11667982692808293
ndcg@10: 0.15130519536593592
ndcg@20: 0.19054979202363959
ndcg@50: 0.25405426550268806
ndcg@100: 0.30978303833625853
